In [42]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType,StringType, FloatType
import pyspark
from pyspark.sql import SparkSession
import os

In [6]:
accessKeyId = "admin"
secretAccessKey = "password"

In [7]:
# spark = (
#     SparkSession.builder
#     .master('spark://spark-iceberg:7077')
#     .appName("Spark Nessie Iceberg Demo")
#     .config("spark.driver.memory", "16g")
#     .config("spark.hadoop.fs.s3a.aws.credentials.provider",
#                  "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")\
#     .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.5.2,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.96.1')
#     .config("spark.hadoop.fs.s3a.access.key", accessKeyId)
#     .config("spark.hadoop.fs.s3a.secret.key", secretAccessKey)
#     .config("spark.hadoop.fs.s3a.path.style.access", True)
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
#     .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
#     .config("spark.sql.catalog.nessie.ref", "main")
#     .config("spark.sql.catalog.nessie.authentication.type", "NONE")
#     .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
#     .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
#     .config("spark.sql.catalog.nessie.warehouse", "s3://warehouse/")
#     .config("fs.s3a.connection.ssl.enabled", "false")
#     .getOrCreate()
# )


In [18]:
spark = (
    SparkSession.builder.appName("Iceberg-Lakehouse-Nessie")
    .config("spark.jars.packages",
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,"
            "software.amazon.awssdk:bundle:2.24.8,"
            "software.amazon.awssdk:url-connection-client:2.24.8")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.lakehouse_prod_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.lakehouse_prod_catalog.type", "nessie")  # <--- use nessie, not rest
    .config("spark.sql.catalog.lakehouse_prod_catalog.uri", "http://nessie:19120/api/v2")  # <--- correct URL
    .config("spark.sql.catalog.lakehouse_prod_catalog.warehouse", "s3://warehouse/wh/")
    .config("spark.sql.catalog.lakehouse_prod_catalog.ref", "main")  # <--- default Nessie branch
    .getOrCreate()
)

25/10/07 14:38:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
+---------+



In [12]:
spark.sql("SHOW NAMESPACES").show()

+---------+
|namespace|
+---------+
+---------+



In [55]:
# spark.sql("CREATE DATABASE IF NOT EXISTS test_db")
# spark.sql("DROP TABLE IF EXISTS test_db.test_table_001")

DataFrame[]

In [19]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS lakehouse_prod_catalog.test_db;").show()

++
||
++
++



In [20]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS lakehouse_prod_catalog.test_db.sample_table (
            id BIGINT,
            name STRING
        ) USING iceberg;
    """
).show()

++
||
++
++



In [21]:
# Insert data into the sample_table
spark.sql(
    """
        INSERT INTO lakehouse_prod_catalog.test_db.sample_table VALUES
        (1, 'Alice'),
        (2, 'Bob');
    """
).show()

++
||
++
++



In [22]:
# Query the data from the table
spark.sql("SELECT * FROM lakehouse_prod_catalog.test_db.sample_table").show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  1|Alice|
|  2|  Bob|
+---+-----+



In [23]:
# Extract a Data
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-04-01 00:00:18|  2021-04-01 00:21:54|            1.0|          8.4|       1.0|                 N|          79|         116|           1|       25.5|  3.0|    0.5|      5.8

In [24]:
df.writeTo("lakehouse_prod_catalog.test_db.test_table_001").createOrReplace()

In [29]:
read_df = spark.read.table("lakehouse_prod_catalog.test_db.test_table_001")
read_df.show(10)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2021-04-01 00:00:18|  2021-04-01 00:21:54|            1.0|          8.4|       1.0|                 N|          79|         116|           1|       25.5|  3.0|    0.5|      5.8

In [31]:
# df.write.mode("overwrite").format("parquet").saveAsTable("test_db.test_table_001")

In [25]:
# # Try with Nessie Catalog
# # df.write.mode("overwrite").format("iceberg").saveAsTable("nessie.test_db.test_table_002")
# df.writeTo("test_db.test_table_002").using("iceberg").createOrReplace()

In [26]:
# Show Catalog
spark.sql("SHOW CATALOGS").show()

+--------------------+
|             catalog|
+--------------------+
|lakehouse_prod_ca...|
|       spark_catalog|
+--------------------+



In [27]:
# spark.sql("SHOW NAMESPACES IN lakehouse_prod_catalog").show()

In [17]:
spark.stop()